In [1]:
# importing helper functions
from data import *

In [12]:
# importing river
import numbers
from river import compose
from river import datasets
from river import evaluate
from river import linear_model
from river import neighbors
from river import metrics
from river import preprocessing
from river import neural_net
from river import ensemble
from river import tree
from river import stream
import graphviz

In [3]:
from tqdm import tqdm

In [4]:
# import preprocessing tools
import numpy as np
import pandas as pd

In [6]:
# dataset creation
def make_target(df, h):
    if h >= 1:
        y = df.shift(periods=-h, freq='1H')["pc"]
        df.pop('pc15')
        df.pop('pc30')
        df.pop('pc45')
        y = y[h:]
        X = df[:-h]
    else: 
        y = {
            0.25: df["pc15"],
            0.5: df["pc30"],
            0.75: df["pc45"]
        }[h]
        df.pop('pc15')
        df.pop('pc30')
        df.pop('pc45')        
        X = df
    return X, y

# load data
static_df = load_static()
weather_df = pd.read_pickle('data/weather/weather.pkl')

In [21]:
sensors = [1, 24, 35, 43, 44, 68, 103, 105, 106, 108, 109, 110, 116, 117, 122, 124, 144, 145, 157, 168, 177,
 179, 223, 225, 236, 239, 243, 245, 249, 253, 257, 259, 262, 285, 286, 290, 293, 301, 303, 312, 333, 358, 359, 362,
 365, 367, 377, 378, 379, 382, 384, 385, 388, 389, 391, 392, 394, 395]
horizons = [0.25, 0.5, 0.75, 1, 3, 6, 12, 24, 36]

models1 = [
    ('LinReg', 'linear_model.LinearRegression()'),
    ('KNN', 'neighbors.KNNRegressor()'),
    ('HT', 'tree.HoeffdingTreeRegressor(grace_period=20, leaf_prediction="adaptive")'),
    ('HAT', 'tree.HoeffdingAdaptiveTreeRegressor(grace_period=20, leaf_prediction="adaptive")'),
    ('ARF', 'ensemble.AdaptiveRandomForestRegressor(grace_period=20, leaf_prediction="adaptive")')    
]

models15 = [
    ('LinReg', 'linear_model.LinearRegression()'),
    ('KNN', 'neighbors.KNNRegressor()'),
    ('HT', 'tree.HoeffdingTreeRegressor(grace_period=20, leaf_prediction="adaptive", leaf_model=neighbors.KNNRegressor())'),
    ('HAT', 'tree.HoeffdingAdaptiveTreeRegressor(grace_period=20, leaf_prediction="adaptive", leaf_model=neighbors.KNNRegressor())'),
    ('ARF', 'ensemble.AdaptiveRandomForestRegressor(grace_period=20, leaf_prediction="adaptive", leaf_model=neighbors.KNNRegressor())')
]

result = []
# create progress bar
pbar = tqdm(total=50)

for s in sensors:
    for h in horizons:
        # use different models for 15-minute predictions
        models = models15
        if h == 0.25:
            models = models15
        for name, model in models: 
            output = "s{}h{}-{}".format(s, h, name)
            #print(output)

            # load sensor - last parameter is id
            final_df = make_final(static_df, weather_df, s)

            # make dataset
            X_df, y_df = make_target(final_df, h)

            hh = h
            if (hh < 1):
                hh = 1
            selected_features = get_features_sh(s, hh)
            X = X_df[selected_features]
            y = y_df

            dataset = stream.iter_pandas(X, y)            
            reg = eval(model)
            
            metric08 = metrics.R2()
            metric = metrics.R2()
            metricS08 = metrics.SMAPE()
            metricS = metrics.SMAPE()

            i = 0
            step = len(y) / 50

            pbar.reset()

            partial_result = { "sensor": s, "horizon": h, "model": name, "smape": [], "smape08": [], "r2": [], "r208": []}
                      
            for xi, yi in dataset:    
                i = i + 1    
                yp = reg.predict_one(xi)    
                reg.learn_one(xi, yi)      

                # update metrics
                metric.update(yi, yp)
                metricS.update(yi, yp)
                if i > len(y) * 0.8:
                    metric08.update(yi, yp)
                    metricS08.update(yi, yp)

                if i % step < 1:
                    pbar.update(1)
                    pbar.set_description("Sensor {}: h = {}, m = {}, R2 = {:.4f}".format(s, h, name, metric.get()))

                    partial_result["smape"].append(metricS.get())
                    partial_result["smape08"].append(metricS08.get())
                    partial_result["r2"].append(metric.get())
                    partial_result["r208"].append(metric08.get())

            pbar.update(1)
            pbar.set_description("Sensor {}: h = {}, m = {}, R2 = {:.4f}".format(s, h, name, metric.get()))
            partial_result["smape"].append(metricS.get())
            partial_result["smape08"].append(metricS08.get())
            partial_result["r2"].append(metric.get())
            partial_result["r208"].append(metric08.get())

            result.append(partial_result)

pbar.close()


Sensor 243: h = 0.25, m = LinReg, R2 = -71252759768845668700913664.0000:  82%|██████▌ | 41/50 [00:00<00:00, 356.44it/s]C:\Users\klemen\AppData\Roaming\Python\Python38\site-packages\river\metrics\r2.py:71: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - (self._residual_sum_of_squares / total_sum_of_squares)
Sensor 243: h = 0.25, m = KNN, R2 = 0.5443:  82%|███████████████████████████████▏      | 41/50 [00:00<00:00, 66.18it/s]C:\Users\klemen\AppData\Roaming\Python\Python38\site-packages\river\metrics\r2.py:71: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (self._residual_sum_of_squares / total_sum_of_squares)
Sensor 395: h = 36, m = ARF, R2 = 0.6959: : 51it [01:40,  1.97s/it]                                                    


In [20]:
pbar.close()

Sensor 1: h = 1, m = ARF, R2 = -45548488179.3880:  80%|█████████████████████████▌      | 40/50 [00:46<00:11,  1.16s/it]


In [22]:
# save features
with open("results/river_st_particular.pkl", "wb") as f:
    pickle.dump(result, f)

In [18]:
result

[{'sensor': 1,
  'horizon': 0.25,
  'model': 'LinReg',
  'smape': [199.99999584616393,
   199.99999791684726,
   199.99999860980705,
   199.99999895680932,
   199.99999916618455,
   199.99999930485038,
   199.999999403976,
   199.99999947834328,
   199.99999953653014,
   199.9999995827846,
   199.999999620634,
   199.99999965157303,
   199.99999967847708,
   199.99999970139967,
   199.99999972126116,
   199.99999973863578,
   199.99999975390608,
   199.9999997674988,
   199.99999977970245,
   199.99999979064393,
   199.99999980065314,
   199.99999980969523,
   199.9999998179515,
   199.9999998255191,
   199.99999983252528,
   199.999999838954,
   199.99999984490518,
   199.99999985043397,
   199.9999998555818,
   199.99999986041433,
   199.99999986490914,
   199.999999869119,
   199.9999998730757,
   199.9999998768233,
   199.9999998803332,
   199.9999998836505,
   199.99999988678613,
   199.9999998897766,
   199.9999998925974,
   199.99999989527663,
   199.99999989782066,
   199.99999